In [1]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, median_absolute_error, r2_score, accuracy_score, precision_score, recall_score

# Abundance

## Sample

In [2]:
baseline = pd.read_csv('analysis_results/baseline_abundances.csv')
estimate_reg = pd.read_csv('analysis_results/estimated_abundances_reg.csv')
estimate_w = pd.read_csv('analysis_results/estimated_abundances_w.csv')
rs = [1, 2, 3, 4, 5]

metrics_reg = pd.DataFrame(columns=['filename', 'r', 'sample_id', 'mae', 'mdae', 'r2'])
metrics_w = pd.DataFrame(columns=['filename', 'r', 'sample_id', 'mae', 'mdae', 'r2'])
for file in baseline['filename'].unique():
    bl = baseline[baseline['filename'] == file]
    bl = bl[['0', '1', '2', '3', '4', '5', '6', '7', '8']].iloc[0].values
    
    es_r = estimate_reg[estimate_reg['filename'] == file]
    es_w = estimate_w[estimate_w['filename'] == file]
    
    for r in rs:
        es_r_r = es_r[es_r['r'] == r]
        es_w_r = es_w[es_w['r'] == r]
        
        for i in range(len(es_r_r['sample_id'].unique())):
            es_r_ab = es_r_r[es_r_r['sample_id'] == i][['0', '1', '2', '3', '4', '5', '6', '7', '8']].iloc[0].values
            temp = {'filename': file, 'r': r, 'sample_id': i, 'mae': mean_absolute_error(bl, es_r_ab), 'mdae': median_absolute_error(bl, es_r_ab), 'r2': r2_score(bl, es_r_ab)}
            metrics_reg = pd.concat([metrics_reg, pd.DataFrame([temp])])
            
        for i in range(len(es_w_r['sample_id'].unique())):
            es_w_ab = es_w_r[es_w_r['sample_id'] == i][['0', '1', '2', '3', '4', '5', '6', '7', '8']].iloc[0].values
            temp = {'filename': file, 'r': r, 'sample_id': i, 'mae': mean_absolute_error(bl, es_w_ab), 'mdae': median_absolute_error(bl, es_w_ab), 'r2': r2_score(bl, es_w_ab)}
            metrics_w = pd.concat([metrics_w, pd.DataFrame([temp])])

metrics_w.to_csv('abundances_sample_w.csv', index=False)
metrics_reg.to_csv('abundances_sample_reg.csv', index=False)

/tmp/ipykernel_119484/2427802297.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_reg = pd.concat([metrics_reg, pd.DataFrame([temp])])
/tmp/ipykernel_119484/2427802297.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_w = pd.concat([metrics_w, pd.DataFrame([temp])])


## Plot

In [3]:
baseline = pd.read_csv('analysis_results/baseline_abundances.csv')
estimate_reg = pd.read_csv('analysis_results/estimated_abundances_reg.csv')
estimate_w = pd.read_csv('analysis_results/estimated_abundances_w.csv')
rs = [1, 2, 3, 4, 5]

metrics_reg = pd.DataFrame(columns=['filename', 'r', 'mae', 'mdae', 'r2'])
metrics_w = pd.DataFrame(columns=['filename', 'r', 'mae', 'mdae', 'r2'])
for file in baseline['filename'].unique():
    bl = baseline[baseline['filename'] == file]
    bl = bl[['0', '1', '2', '3', '4', '5', '6', '7', '8']].iloc[0].values
    
    es_r = estimate_reg[estimate_reg['filename'] == file]
    es_w = estimate_w[estimate_w['filename'] == file]
    
    for r in rs:
        es_r_r = es_r[es_r['r'] == r]
        es_w_r = es_w[es_w['r'] == r]
        
        es_r_ab = es_r_r[['0', '1', '2', '3', '4', '5', '6', '7', '8']].sum().values
        es_r_ab /= len(es_r_r['sample_id'].unique())
        
        temp = {'filename': file, 'r': r, 'mae': mean_absolute_error(bl, es_r_ab), 'mdae': median_absolute_error(bl, es_r_ab), 'r2': r2_score(bl, es_r_ab)}
        metrics_reg = pd.concat([metrics_reg, pd.DataFrame([temp])])
        
        es_w_ab = es_w_r[['0', '1', '2', '3', '4', '5', '6', '7', '8']].sum().values
        es_w_ab /= len(es_w_r['sample_id'].unique())
        
        temp = {'filename': file, 'r': r, 'mae': mean_absolute_error(bl, es_w_ab), 'mdae': median_absolute_error(bl, es_w_ab), 'r2': r2_score(bl, es_w_ab)}
        metrics_w = pd.concat([metrics_w, pd.DataFrame([temp])])

metrics_w.to_csv('abundances_plot_w.csv', index=False)
metrics_reg.to_csv('abundances_plot_reg.csv', index=False)

/tmp/ipykernel_119484/2988431089.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_reg = pd.concat([metrics_reg, pd.DataFrame([temp])])
/tmp/ipykernel_119484/2988431089.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_w = pd.concat([metrics_w, pd.DataFrame([temp])])


## Initial Location

In [4]:
baseline = pd.read_csv('analysis_results/baseline_abundances.csv')
estimate_reg = pd.read_csv('analysis_results/estimated_abundances_reg.csv')
estimate_w = pd.read_csv('analysis_results/estimated_abundances_w.csv')
rs = [1, 2, 3, 4, 5]

metrics_reg = pd.DataFrame(columns=['filename', 'r', 'mae', 'mdae', 'r2'])
metrics_w = pd.DataFrame(columns=['filename', 'r', 'mae', 'mdae', 'r2'])

filenames = baseline['filename'].unique()
for files in [filenames[:len(filenames)//2], filenames[len(filenames)//2:]]:
    bl = baseline[baseline['filename'].isin(files)]
    bl = bl[['0', '1', '2', '3', '4', '5', '6', '7', '8']].sum().values
    bl /= len(files)
    
    es_r = estimate_reg[estimate_reg['filename'].isin(files)]
    es_w = estimate_w[estimate_w['filename'].isin(files)]
    
    for r in rs:
        es_r_r = es_r[es_r['r'] == r]
        es_w_r = es_w[es_w['r'] == r]
        
        es_r_ab = es_r_r[['0', '1', '2', '3', '4', '5', '6', '7', '8']].sum().values
        es_r_ab /= len(es_r_r['sample_id'].unique())
        es_r_ab /= len(files)
        
        temp = {'filename': files, 'r': r, 'mae': mean_absolute_error(bl, es_r_ab), 'mdae': median_absolute_error(bl, es_r_ab), 'r2': r2_score(bl, es_r_ab)}
        metrics_reg = pd.concat([metrics_reg, pd.DataFrame([temp])])
        
        es_w_ab = es_w_r[['0', '1', '2', '3', '4', '5', '6', '7', '8']].sum().values
        es_w_ab /= len(es_w_r['sample_id'].unique())
        es_w_ab /= len(files)
        
        temp = {'filename': files, 'r': r, 'mae': mean_absolute_error(bl, es_w_ab), 'mdae': median_absolute_error(bl, es_w_ab), 'r2': r2_score(bl, es_w_ab)}
        metrics_w = pd.concat([metrics_w, pd.DataFrame([temp])])
        
metrics_w.to_csv('abundances_loc_w.csv', index=False)
metrics_reg.to_csv('abundances_loc_reg.csv', index=False)

/tmp/ipykernel_119484/1237149562.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_reg = pd.concat([metrics_reg, pd.DataFrame([temp])])
/tmp/ipykernel_119484/1237149562.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_w = pd.concat([metrics_w, pd.DataFrame([temp])])


# Diversity

## Sample

In [5]:
baseline = pd.read_csv('analysis_results/baseline_diversity.csv')
estimate_reg = pd.read_csv('analysis_results/estimated_diversity_reg.csv')
estimate_w = pd.read_csv('analysis_results/estimated_diversity_w.csv')
rs = [1, 2, 3, 4, 5]

metrics_reg = pd.DataFrame(columns=['filename', 'r', 'sample_id', 'acc', 'prec', 'rec'])
metrics_w = pd.DataFrame(columns=['filename', 'r', 'sample_id', 'acc', 'prec', 'rec'])
for file in baseline['filename'].unique():
    bl = baseline[baseline['filename'] == file]
    bl = bl[['0', '1', '2', '3', '4', '5', '6', '7', '8']].iloc[0].values.astype(int)
    
    es_r = estimate_reg[estimate_reg['filename'] == file]
    es_w = estimate_w[estimate_w['filename'] == file]
    
    for r in rs:
        es_r_r = es_r[es_r['r'] == r]
        es_w_r = es_w[es_w['r'] == r]
        
        for i in range(len(es_r_r['sample_id'].unique())):
            es_r_ab = es_r_r[es_r_r['sample_id'] == i][['0', '1', '2', '3', '4', '5', '6', '7', '8']].iloc[0].values.astype(int)
            temp = {'filename': file, 'r': r, 'sample_id': i, 'acc': accuracy_score(bl, es_r_ab), 'prec': precision_score(bl, es_r_ab), 'rec': recall_score(bl, es_r_ab)}
            metrics_reg = pd.concat([metrics_reg, pd.DataFrame([temp])])
            
        for i in range(len(es_w_r['sample_id'].unique())):
            es_w_ab = es_w_r[es_w_r['sample_id'] == i][['0', '1', '2', '3', '4', '5', '6', '7', '8']].iloc[0].values.astype(int)
            temp = {'filename': file, 'r': r, 'sample_id': i, 'acc': accuracy_score(bl, es_w_ab), 'prec': precision_score(bl, es_w_ab), 'rec': recall_score(bl, es_w_ab)}
            metrics_w = pd.concat([metrics_w, pd.DataFrame([temp])])

metrics_w.to_csv('diversity_sample_w.csv', index=False)
metrics_reg.to_csv('diversity_sample_reg.csv', index=False)

/tmp/ipykernel_119484/4076057136.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_reg = pd.concat([metrics_reg, pd.DataFrame([temp])])
/home/timovdk/soil_sim/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/timovdk/soil_sim/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, m

## Plot

In [6]:
baseline = pd.read_csv('analysis_results/baseline_diversity.csv')
estimate_reg = pd.read_csv('analysis_results/estimated_diversity_reg.csv')
estimate_w = pd.read_csv('analysis_results/estimated_diversity_w.csv')
rs = [1, 2, 3, 4, 5]

metrics_reg = pd.DataFrame(columns=['filename', 'r', 'acc', 'prec', 'rec'])
metrics_w = pd.DataFrame(columns=['filename', 'r', 'acc', 'prec', 'rec'])
for file in baseline['filename'].unique():
    bl = baseline[baseline['filename'] == file]
    bl = bl[['0', '1', '2', '3', '4', '5', '6', '7', '8']].iloc[0].values.astype(bool).astype(int)
    
    es_r = estimate_reg[estimate_reg['filename'] == file]
    es_w = estimate_w[estimate_w['filename'] == file]
    
    for r in rs:
        es_r_r = es_r[es_r['r'] == r]
        es_w_r = es_w[es_w['r'] == r]
        
        es_r_ab = es_r_r[['0', '1', '2', '3', '4', '5', '6', '7', '8']].sum().values.astype(bool).astype(int)
        temp = {'filename': file, 'r': r, 'acc': accuracy_score(bl, es_r_ab), 'prec': precision_score(bl, es_r_ab), 'rec': recall_score(bl, es_r_ab)}
        metrics_reg = pd.concat([metrics_reg, pd.DataFrame([temp])])
        
        es_w_ab = es_w_r[['0', '1', '2', '3', '4', '5', '6', '7', '8']].sum().astype(bool).astype(int)
        temp = {'filename': file, 'r': r, 'acc': accuracy_score(bl, es_w_ab), 'prec': precision_score(bl, es_w_ab), 'rec': recall_score(bl, es_w_ab)}
        metrics_w = pd.concat([metrics_w, pd.DataFrame([temp])])

metrics_w.to_csv('diversity_plot_w.csv', index=False)
metrics_reg.to_csv('diversity_plot_reg.csv', index=False)

/tmp/ipykernel_119484/133958993.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_reg = pd.concat([metrics_reg, pd.DataFrame([temp])])
/tmp/ipykernel_119484/133958993.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_w = pd.concat([metrics_w, pd.DataFrame([temp])])


## Initial Location

In [7]:
baseline = pd.read_csv('analysis_results/baseline_diversity.csv')
estimate_reg = pd.read_csv('analysis_results/estimated_diversity_reg.csv')
estimate_w = pd.read_csv('analysis_results/estimated_diversity_w.csv')
rs = [1, 2, 3, 4, 5]

metrics_reg = pd.DataFrame(columns=['filename', 'r', 'acc', 'prec', 'rec'])
metrics_w = pd.DataFrame(columns=['filename', 'r', 'acc', 'prec', 'rec'])

filenames = baseline['filename'].unique()
for files in [filenames[:len(filenames)//2], filenames[len(filenames)//2:]]:
    bl = baseline[baseline['filename'].isin(files)]
    bl = bl[['0', '1', '2', '3', '4', '5', '6', '7', '8']].sum().values.astype(bool).astype(int)
    
    es_r = estimate_reg[estimate_reg['filename'].isin(files)]
    es_w = estimate_w[estimate_w['filename'].isin(files)]
    
    for r in rs:
        es_r_r = es_r[es_r['r'] == r]
        es_w_r = es_w[es_w['r'] == r]
        
        es_r_ab = es_r_r[['0', '1', '2', '3', '4', '5', '6', '7', '8']].sum().values.astype(bool).astype(int)
        temp = {'filename': files, 'r': r, 'acc': accuracy_score(bl, es_r_ab), 'prec': precision_score(bl, es_r_ab), 'rec': recall_score(bl, es_r_ab)}
        metrics_reg = pd.concat([metrics_reg, pd.DataFrame([temp])])
        
        es_w_ab = es_w_r[['0', '1', '2', '3', '4', '5', '6', '7', '8']].sum().values.astype(bool).astype(int)
        temp = {'filename': files, 'r': r, 'acc': accuracy_score(bl, es_w_ab), 'prec': precision_score(bl, es_w_ab), 'rec': recall_score(bl, es_w_ab)}
        metrics_w = pd.concat([metrics_w, pd.DataFrame([temp])])
        
metrics_w.to_csv('diversity_loc_w.csv', index=False)
metrics_reg.to_csv('diversity_loc_reg.csv', index=False)

/tmp/ipykernel_119484/769268982.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_reg = pd.concat([metrics_reg, pd.DataFrame([temp])])
/tmp/ipykernel_119484/769268982.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_w = pd.concat([metrics_w, pd.DataFrame([temp])])


# D-Index

## Sample

In [9]:
baseline = pd.read_csv('analysis_results/baseline_d_index.csv')
estimate_reg = pd.read_csv('analysis_results/estimated_d_index_sample_reg.csv')
estimate_w = pd.read_csv('analysis_results/estimated_d_index_sample_w.csv')
rs = [1, 2, 3, 4, 5]

metrics_reg = pd.DataFrame(columns=['filename', 'r', 'sample_id', 'type_id', 'mae', 'mdae', 'r2'])
metrics_w = pd.DataFrame(columns=['filename', 'r', 'sample_id', 'type_id', 'mae', 'mdae', 'r2'])
for file in baseline['filename'].unique():
    bl_f = baseline[baseline['filename'] == file]
    es_r = estimate_reg[estimate_reg['filename'] == file]
    es_w = estimate_w[estimate_w['filename'] == file]

    for t in range(len(baseline['type_id'].unique())):
        bl = bl_f[bl_f['type_id'] == t][['0', '1', '2', '3', '4', '5', '6', '7', '8']].iloc[0].values
        es_r_t = es_r[es_r['type_id'] == t]
        es_w_t = es_w[es_w['type_id'] == t]

        for sample_id in range(len(es_r_t['sample_id'].unique())):
            es_r_s = es_r_t[es_r_t['sample_id'] == sample_id]
            
            for r in rs:
                es_r_r = es_r_s[es_r_s['r'] == r][['0', '1', '2', '3', '4', '5', '6', '7', '8']].iloc[0].values

                temp = {'filename': file, 'r': r, 'sample_id': sample_id,'type_id': t, 'mae': mean_absolute_error(bl, es_r_r), 'mdae': median_absolute_error(bl, es_r_r), 'r2': r2_score(bl, es_r_r)}
                metrics_reg = pd.concat([metrics_reg, pd.DataFrame([temp])])
        
        for sample_id in range(len(es_w_t['sample_id'].unique())):
            es_w_s = es_w_t[es_w_t['sample_id'] == sample_id]
            
            for r in rs:
                es_w_r = es_w_s[es_w_s['r'] == r][['0', '1', '2', '3', '4', '5', '6', '7', '8']].iloc[0].values

                temp = {'filename': file, 'r': r, 'sample_id': sample_id, 'type_id': t, 'mae': mean_absolute_error(bl, es_w_r), 'mdae': median_absolute_error(bl, es_w_r), 'r2': r2_score(bl, es_w_r)}
                metrics_w = pd.concat([metrics_w, pd.DataFrame([temp])])
                
metrics_w.to_csv('d_index_sample_w.csv', index=False)
metrics_reg.to_csv('d_index_sample_reg.csv', index=False)

/tmp/ipykernel_119484/2108794396.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_reg = pd.concat([metrics_reg, pd.DataFrame([temp])])
/tmp/ipykernel_119484/2108794396.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_w = pd.concat([metrics_w, pd.DataFrame([temp])])


## Plot

In [10]:
baseline = pd.read_csv('analysis_results/baseline_d_index.csv')
estimate_reg = pd.read_csv('analysis_results/estimated_d_index_plot_reg.csv')
estimate_w = pd.read_csv('analysis_results/estimated_d_index_plot_w.csv')
rs = [1, 2, 3, 4, 5]

metrics_reg = pd.DataFrame(columns=['filename', 'r', 'type_id', 'mae', 'mdae', 'r2'])
metrics_w = pd.DataFrame(columns=['filename', 'r', 'type_id', 'mae', 'mdae', 'r2'])
for file in baseline['filename'].unique():
    bl_f = baseline[baseline['filename'] == file]
    es_r = estimate_reg[estimate_reg['filename'] == file]
    es_w = estimate_w[estimate_w['filename'] == file]
    
    for t in range(len(baseline['type_id'].unique())):
        bl = bl_f[bl_f['type_id'] == t][['0', '1', '2', '3', '4', '5', '6', '7', '8']].iloc[0].values
        es_r_t = es_r[es_r['type_id'] == t]
        es_w_t = es_w[es_w['type_id'] == t]

        for r in rs:
            es_r_r = es_r_t[es_r_t['r'] == r][['0', '1', '2', '3', '4', '5', '6', '7', '8']].iloc[0].values
            es_w_r = es_w_t[es_w_t['r'] == r][['0', '1', '2', '3', '4', '5', '6', '7', '8']].iloc[0].values

            temp = {'filename': file, 'r': r, 'type_id': t, 'mae': mean_absolute_error(bl, es_r_r), 'mdae': median_absolute_error(bl, es_r_r), 'r2': r2_score(bl, es_r_r)}
            metrics_reg = pd.concat([metrics_reg, pd.DataFrame([temp])])

            temp = {'filename': file, 'r': r, 'type_id': t, 'mae': mean_absolute_error(bl, es_w_r), 'mdae': median_absolute_error(bl, es_w_r), 'r2': r2_score(bl, es_w_r)}
            metrics_w = pd.concat([metrics_w, pd.DataFrame([temp])])

metrics_w.to_csv('d_index_plot_w.csv', index=False)
metrics_reg.to_csv('d_index_plot_reg.csv', index=False)

/tmp/ipykernel_119484/3693277013.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_reg = pd.concat([metrics_reg, pd.DataFrame([temp])])
/tmp/ipykernel_119484/3693277013.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_w = pd.concat([metrics_w, pd.DataFrame([temp])])


## Initial Location

In [11]:
baseline = pd.read_csv('analysis_results/baseline_d_index.csv')
estimate_reg = pd.read_csv('analysis_results/estimated_d_index_plot_reg.csv')
estimate_w = pd.read_csv('analysis_results/estimated_d_index_plot_w.csv')
rs = [1, 2, 3, 4, 5]

metrics_reg = pd.DataFrame(columns=['filename', 'r', 'type_id', 'mae', 'mdae', 'r2'])
metrics_w = pd.DataFrame(columns=['filename', 'r', 'type_id', 'mae', 'mdae', 'r2'])

filenames = baseline['filename'].unique()
for files in [filenames[:len(filenames)//2], filenames[len(filenames)//2:]]:
    bl = baseline[baseline['filename'].isin(files)]
    bl = bl[['0', '1', '2', '3', '4', '5', '6', '7', '8']].sum().values
    bl /= len(files)
    
    es_r = estimate_reg[estimate_reg['filename'].isin(files)]
    es_w = estimate_w[estimate_w['filename'].isin(files)]
    
    for t in range(len(baseline['type_id'].unique())):
        bl = bl_f[bl_f['type_id'] == t][['0', '1', '2', '3', '4', '5', '6', '7', '8']].iloc[0].values
        es_r_t = es_r[es_r['type_id'] == t]
        es_w_t = es_w[es_w['type_id'] == t]

        for r in rs:
            es_r_r = es_r_t[es_r_t['r'] == r][['0', '1', '2', '3', '4', '5', '6', '7', '8']].sum().values
            es_w_r = es_w_t[es_w_t['r'] == r][['0', '1', '2', '3', '4', '5', '6', '7', '8']].sum().values
            
            es_r_r /= len(files)
            es_w_r /= len(files)

            temp = {'filename': files, 'r': r, 'type_id': t, 'mae': mean_absolute_error(bl, es_r_r), 'mdae': median_absolute_error(bl, es_r_r), 'r2': r2_score(bl, es_r_r)}
            metrics_reg = pd.concat([metrics_reg, pd.DataFrame([temp])])

            temp = {'filename': files, 'r': r, 'type_id': t, 'mae': mean_absolute_error(bl, es_w_r), 'mdae': median_absolute_error(bl, es_w_r), 'r2': r2_score(bl, es_w_r)}
            metrics_w = pd.concat([metrics_w, pd.DataFrame([temp])])

metrics_w.to_csv('d_index_loc_w.csv', index=False)
metrics_reg.to_csv('d_index_loc_reg.csv', index=False)

/tmp/ipykernel_119484/2235105670.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_reg = pd.concat([metrics_reg, pd.DataFrame([temp])])
/tmp/ipykernel_119484/2235105670.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_w = pd.concat([metrics_w, pd.DataFrame([temp])])
